# Lecture 2: Introduction to Feature Engineering

## Setup

In [2]:
import numpy as np 
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [3]:
# articles = pd.read_csv('data/articles.csv')
# customers = pd.read_csv('data/customers.csv')
# transactions = pd.read_csv('data/transactions_train.csv')

## The H&M Dataset

In [4]:
# articles.info()

In [5]:
# customers.info()

In [6]:
# transactions.info()

In [7]:
# X = transactions.merge(customers, how='inner', on='customer_id')
# X = X.merge(articles, how='inner', on='article_id')

### Creating Samples 
If you would rather work with samples instead of the whole dataset (while prototyping your code). You can use the code below:

In [8]:
# Adapted from: https://www.kaggle.com/code/paweljankiewicz/hm-create-dataset-samples
# This extracts three sampled datasets, containing 0.1%, 1% and 5% of all users and their transactions, and the associated articles.
# for sample_repr, sample in [("01", 0.001), ("1", 0.01), ("5", 0.05)]:
#     customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False)
#     customers_sample_ids = set(customers_sample["customer_id"])
#     transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
#     articles_sample_ids = set(transactions_sample["article_id"])
#     articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
#     customers_sample.to_csv(f"data/customers_sample{sample_repr}.csv.gz", index=False)
#     transactions_sample.to_csv(f"data/transactions_sample{sample_repr}.csv.gz", index=False)
#     articles_sample.to_csv(f"data/articles_sample{sample_repr}.csv.gz", index=False)

In [9]:
# articles_sample = pd.read_csv('data/articles_sample5.csv.gz')
# customers_sample = pd.read_csv('data/customers_sample5.csv.gz')
# transactions_sample = pd.read_csv('data/transactions_sample5.csv.gz')

## A Simplified Task: Binary Classification

The task of predicting which 12 items users are most likely to buy in the next week is difficult to translate to a traditional classification machine learning setting. 
To obtain the 12 items a user is most likely to buy, we need to make predictions for all items (or the ones selected by a baseline) and select the 12 that have the highest predicted scores.

In this assignment, we'll consider a simplified task: Predict whether a user ordered a single item or not, based on the features of the user and the item. 
We provide a baseline logistic regression model below, but haven't done much feature preprocessing or engineering!
Initially, it is always best to focus your efforts on getting your features in the right shape and setting up the right validation scheme and baselines.
Once you are sure that your features add value and your validation scheme is correct, then you typically move on to trying more elaborate models.

### Creating the Dataset

In [10]:
# If you'd rather use a sample. Uncomment the following code:
sample = "5"

articles = pd.read_csv(f'data/articles_sample{sample}.csv.gz')
customers = pd.read_csv(f'data/customers_sample{sample}.csv.gz')
transactions = pd.read_csv(f'data/transactions_sample{sample}.csv.gz')

In [11]:
transactions['ordered'] = 1

The problem setting is an example of a "PU learning" problem, i.e. only positives are labeled, everything else is unlabeled (and can be either positive or negative). 
Of course, we cannot train a classifier with just positive samples: The classifier will just learn that everything is positive.
Therefore, we need to manually generate negative samples.

Below, we use a simple random negative sampling strategy.
We want to create a balanced dataset, meaning that we have just as many positives as negatives.
This makes sure that the classifier will not benefit from predicting the positive/negative class more often than the other.
Realistically, the amount of positive samples is of course many times smaller than the amount of unlabeled, possibly negative instances.


If you want to try your hand at a more complex negative sampling strategy, you may want to check out this blog as a starting point: https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7.



In [12]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,001fd23db1109a94bba1319bb73df0b479059027c182da...,631744002,0.016932,1,1
1,2018-09-20,001fd23db1109a94bba1319bb73df0b479059027c182da...,562252035,0.021593,1,1
2,2018-09-20,00708c3da4d07706d4cad77c6aecc1b1ce33d21d73022c...,255396006,0.067780,2,1
3,2018-09-20,00708c3da4d07706d4cad77c6aecc1b1ce33d21d73022c...,594834010,0.042356,2,1
4,2018-09-20,00708c3da4d07706d4cad77c6aecc1b1ce33d21d73022c...,516712001,0.022017,2,1


In [13]:
# What's happening here? 
# We're creating negative samples. I.e. we're creating transactions that didn't actually occur.
# First, we need to know which interactions did occur:
positive_pairs = list(map(tuple, transactions[['customer_id', 'article_id']].drop_duplicates().values))

In [14]:
# Then we need to know what every synthetic transaction should contain: a date, a customer_id, an article_id, price, sales_channel_id. We will set ordered = 0, as these transactions didn't really occur.
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,001fd23db1109a94bba1319bb73df0b479059027c182da...,631744002,0.016932,1,1
1,2018-09-20,001fd23db1109a94bba1319bb73df0b479059027c182da...,562252035,0.021593,1,1
2,2018-09-20,00708c3da4d07706d4cad77c6aecc1b1ce33d21d73022c...,255396006,0.067780,2,1
3,2018-09-20,00708c3da4d07706d4cad77c6aecc1b1ce33d21d73022c...,594834010,0.042356,2,1
4,2018-09-20,00708c3da4d07706d4cad77c6aecc1b1ce33d21d73022c...,516712001,0.022017,2,1


In [15]:
# Extract real values
real_dates = transactions["t_dat"].unique()
real_customers = transactions["customer_id"].unique()
real_articles = transactions["article_id"].unique()
real_channels = transactions["sales_channel_id"].unique()
article_and_price = transactions[["article_id","price"]].drop_duplicates("article_id").set_index("article_id").squeeze()

In [16]:
# How many negatives do we need to sample?
num_neg_pos = transactions.shape[0]
print(num_neg_pos)

1566354


In [17]:
# Sampling negatives by selecting random users, articles, dates and sales channel:
# Note: This is quite naive. Some articles may not even have been available at the date we are sampling.
random.seed(42)

# Afterwards, we need to remove potential duplicates, so we'll sample too many.
num_neg_samples = int(num_neg_pos * 1.1)

# Sample each of the independent attributes.
neg_dates = np.random.choice(real_dates, size=num_neg_samples)
neg_articles = np.random.choice(real_articles, size=num_neg_samples)
neg_customers = np.random.choice(real_customers, size=num_neg_samples)
neg_channels = np.random.choice(real_channels, size=num_neg_samples)
ordered = np.array([0] * num_neg_samples)
# Assign to every article a real price.
neg_prices = article_and_price[neg_articles].values

In [18]:
neg_transactions = pd.DataFrame([neg_dates, neg_customers, neg_articles, neg_prices, neg_channels, ordered], index=transactions.columns).T

In [19]:
# Result:
neg_transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2020-06-24,00386204792883fdad2a2f5aa6ee2354e6019518fe31cc...,720137007,0.003373,1,0
1,2020-07-25,6e28d610e006e71d4708469df683efcc1bbed528f73945...,399223034,0.033881,2,0
2,2018-10-17,eb84b0d215fff1bd481867543bb14f5cda104904b07eae...,669900002,0.040661,1,0
3,2019-11-21,c2e7fc9f53ccce64884ff3b65a3cbef869ebcceeb04468...,679135003,0.042356,2,0
4,2019-06-23,6dc70e9b92075149ef1ac4696404775e21ec6908ab1a53...,799679001,0.010593,2,0


In [20]:
neg_transactions.shape

(1722989, 6)

In [21]:
# Remove random negative samples that actually coincide with positives
df = neg_transactions[
    ~neg_transactions.set_index(["customer_id", "article_id"]).index.isin(positive_pairs)
]

# Remove any excess
chosen_neg_transactions = df.sample(num_neg_pos)

In [22]:
# Concat the negative samples to the positive samples:
transactions = pd.concat([transactions, chosen_neg_transactions])
transactions = transactions.merge(customers, how="inner", on='customer_id')
transactions = transactions.merge(articles, how="inner", on='article_id')

In [23]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3132708 entries, 0 to 3132707
Data columns (total 36 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   t_dat                         object 
 1   customer_id                   object 
 2   article_id                    object 
 3   price                         object 
 4   sales_channel_id              object 
 5   ordered                       object 
 6   FN                            float64
 7   Active                        float64
 8   club_member_status            object 
 9   fashion_news_frequency        object 
 10  age                           float64
 11  postal_code                   object 
 12  product_code                  int64  
 13  prod_name                     object 
 14  product_type_no               int64  
 15  product_type_name             object 
 16  product_group_name            object 
 17  graphical_appearance_no       int64  
 18  graphical_appearance_n

### Basic Preprocessing
Some very basic preprocessing.

In [24]:
# I'm dropping a lot of columns, use them in your engineering tasks!
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered']].copy()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,001fd23db1109a94bba1319bb73df0b479059027c182da...,53.0,631744002,1,0.016932,1
1,ab355a515d8eb0038a4b39aa9fd9439af221d3beca0794...,27.0,631744002,1,0.016932,0
2,4471add869b42428fcc4aab3dccdac378542f4c2ec4b15...,24.0,631744002,2,0.016932,0
3,02fcbfadd7f268ee2604ae1d2b8d944e2e946f8b297cbd...,26.0,631744002,2,0.016932,0
4,c5609385a4ab538a56c5e1c54355c5b4e6fa5956d3ce9d...,47.0,631744002,2,0.016932,0


In [25]:
# Does it make sense to label encode?
# Label encoding the customer and article IDs:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()

In [26]:
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [27]:
# If you want to go back to the original encoding:
customer_encoder.inverse_transform([2])

array(['0001b0127d3e5ff8dadcfc6e5043682dba2070f2667081623faeb31c996242a6'],
      dtype=object)

In [28]:
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,32,53.0,19551,1,0.016932,1
1,45430,27.0,19551,1,0.016932,0
2,18287,24.0,19551,2,0.016932,0
3,796,26.0,19551,2,0.016932,0
4,52507,47.0,19551,2,0.016932,0


In [29]:
# Can you come up with a NaN strategy that makes sense for each column in the dataset?
# Imputing all NaN values with zeros:
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()

False

In [30]:
# Does it make sense to one-hot encode?
# One-hot-encoding sales_channel_id:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id'])

In [31]:
transactions_processed.head()

,customer_id,age,article_id,price,ordered,sales_channel_id_1,sales_channel_id_2
0,32,53.0,19551,0.016932,1,1,0
1,45430,27.0,19551,0.016932,0,1,0
2,18287,24.0,19551,0.016932,0,0,1
3,796,26.0,19551,0.016932,0,0,1
4,52507,47.0,19551,0.016932,0,0,1


In [32]:
# Creating a Train / Test Split:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)

In [33]:
X_train.head()

,customer_id,age,article_id,price,sales_channel_id_1,sales_channel_id_2
176388,1839,20.0,22627,0.016932,1,0
806818,17597,27.0,53004,0.118627,1,0
1807634,32964,0.0,62961,0.013542,0,1
1343423,23362,57.0,10439,0.033881,0,1
233105,42647,27.0,13494,0.015237,0,1


In [34]:
y_train.head()

176388     0
806818     0
1807634    0
1343423    0
233105     0
Name: ordered, dtype: int64

## Baseline Model

In [35]:
# Will take a few minutes to run, if you're using the whole dataset:
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)

In [36]:
baseline.predict_proba(X_test)

array([[0.51234718, 0.48765282],
       [0.50411886, 0.49588114],
       [0.51145939, 0.48854061],
       ...,
       [0.51829938, 0.48170062],
       [0.50760542, 0.49239458],
       [0.49215028, 0.50784972]])

In [37]:
y_test

1649772    1
482687     1
393230     1
3082071    0
2353135    1
          ..
3097538    0
2032851    0
1323787    0
535384     1
2428458    0
Name: ordered, Length: 313271, dtype: int64

In [38]:
# Mean Accuracy:
baseline.score(X_test, y_test)
# As you can see, the accuracy is ~0.51. In other words, the classifier predicts correctly 51% of the time whether a customer did or didn't buy an item.
# Can you improve this baseline logistic regression model by doing better preprocessing and generating new features?
# Also think about my steps! Did it make sense to include the article and customer ids? (And things like that)

0.5064433030826345

In [39]:
# Classification Metrics:
predictions = baseline.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.50      0.55      0.52    156287
           1       0.51      0.47      0.49    156984

    accuracy                           0.51    313271
   macro avg       0.51      0.51      0.51    313271
weighted avg       0.51      0.51      0.51    313271



## Assignment: Feature engineering
**TODO:** 
- In groups (of 2-3 students), think about (a few) features that can be engineered (preprocess and generate new features). Divide the work!
- Do these engineered features improve the baseline model?
- Add your thoughts & results to a slide deck for discussion next week (again, 1 slide per person).


In [40]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered,FN,Active,club_member_status,fashion_news_frequency,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,2018-09-20,001fd23db1109a94bba1319bb73df0b479059027c182da...,631744002,0.016932,1,1,NaN,NaN,ACTIVE,NONE,...,Shopbasket Socks,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Fine-knit shaftless socks with a silicone trim...
1,2019-08-27,ab355a515d8eb0038a4b39aa9fd9439af221d3beca0794...,631744002,0.016932,1,0,1.0,1.0,ACTIVE,Regularly,...,Shopbasket Socks,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Fine-knit shaftless socks with a silicone trim...
2,2019-02-25,4471add869b42428fcc4aab3dccdac378542f4c2ec4b15...,631744002,0.016932,2,0,NaN,NaN,ACTIVE,NONE,...,Shopbasket Socks,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Fine-knit shaftless socks with a silicone trim...
3,2020-06-12,02fcbfadd7f268ee2604ae1d2b8d944e2e946f8b297cbd...,631744002,0.016932,2,0,NaN,NaN,ACTIVE,NONE,...,Shopbasket Socks,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Fine-knit shaftless socks with a silicone trim...
4,2020-01-21,c5609385a4ab538a56c5e1c54355c5b4e6fa5956d3ce9d...,631744002,0.016932,2,0,1.0,1.0,ACTIVE,Regularly,...,Shopbasket Socks,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Fine-knit shaftless socks with a silicone trim...


In [41]:
fn_encoder = preprocessing.LabelEncoder()
active_encoder = preprocessing.LabelEncoder()
club_member_status_encoder = preprocessing.LabelEncoder()
fashion_news_encoder = preprocessing.LabelEncoder()

transactions['FN'] = fn_encoder.fit_transform(transactions['FN'])
transactions['Active'] = active_encoder.fit_transform(transactions['Active'])
transactions['club_member_status'] = club_member_status_encoder.fit_transform(transactions['club_member_status'])
transactions['fashion_news_frequency'] = fashion_news_encoder.fit_transform(transactions['fashion_news_frequency'])

In [42]:
transactions['t_days'] = df['t_dat'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d').value)
article_grouped = transactions.groupby('article_id')
customer_grouped = transactions.groupby('customer_id')

interval = lambda group: group['t_days'].max() - group['t_days'].min() / group['t_days'].count()

mean_age = article_grouped['age'].mean().reset_index(name="mean_age")
last_bought = article_grouped['t_days'].max().reset_index(name="last_bought")
buy_interval = article_grouped.apply(interval).reset_index(name="buy_interval")
mean_price = customer_grouped['price'].mean().reset_index(name="mean_price")

In [43]:
# buy_interval.head()

In [48]:
from sklearn.preprocessing import minmax_scale

transactions.drop(columns=["mean_age", "age_diff", "last_bought", "buy_interval", "mean_price"], axis=1, inplace=True, errors='ignore')

transactions = pd.merge(transactions, mean_age, on='article_id', how='left')
transactions = pd.merge(transactions, last_bought, on='article_id', how='left')
transactions = pd.merge(transactions, buy_interval, on='article_id', how='left')
transactions = pd.merge(transactions, mean_price, on='customer_id', how='left')

transactions['age_diff'] = transactions['age'] - transactions['mean_age']
transactions['price_diff'] = transactions['price'] - transactions['mean_price']

transactions['age_diff'] = pd.DataFrame(minmax_scale(transactions['age_diff']))
transactions['age'] = pd.DataFrame(minmax_scale(transactions['age']))
transactions['last_bought'] = pd.DataFrame(minmax_scale(transactions['last_bought']))
transactions['buy_interval'] = pd.DataFrame(minmax_scale(transactions['buy_interval']))
transactions['price_diff'] = pd.DataFrame(minmax_scale(transactions['price_diff']))
transactions['price'] = pd.DataFrame(minmax_scale(transactions['price']))

In [55]:
# processed = transactions[['sales_channel_id', 'ordered', 'price', 'price_diff', 'age_diff', 'last_bought', 'buy_interval', 'FN', 'Active', 'club_member_status', 'fashion_news_frequency']]
processed = transactions[['sales_channel_id', 'ordered', 'FN', 'Active', 'club_member_status', 'fashion_news_frequency']]

# processed = transactions[['age', 'sales_channel_id', 'price', 'ordered']].copy()

processed = processed.fillna(0)
processed.isnull().values.any()

processed = pd.get_dummies(processed, columns=['sales_channel_id'])

processed.head(10)

,ordered,FN,Active,club_member_status,fashion_news_frequency,sales_channel_id_1,sales_channel_id_2
0,1,1,1,0,1,1,0
1,0,0,0,0,2,1,0
2,0,1,1,0,1,0,1
3,0,1,1,0,1,0,1
4,0,0,0,0,2,0,1
5,0,1,1,0,1,0,1
6,1,1,1,0,1,1,0
7,0,0,0,0,2,0,1
8,0,1,1,0,1,1,0
9,1,0,0,0,2,1,0


In [56]:
X_train, X_test, y_train, y_test = train_test_split(processed.drop('ordered', axis=1), processed['ordered'], test_size=0.10, random_state=42)

baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)

baseline.score(X_test, y_test)

predictions = baseline.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.63      0.54      0.58    156287
           1       0.60      0.68      0.64    156984

    accuracy                           0.61    313271
   macro avg       0.61      0.61      0.61    313271
weighted avg       0.61      0.61      0.61    313271

